# Critical Event System

In [1]:
%matplotlib inline
import os
import sys
import IPython
import sklearn as sk
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import get_data_home, load_files
import patoolib
import shutil
import codecs
from contextlib import suppress

print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('scikit-learn version:', sk.__version__)
print ('matplotlib version:', matplotlib.__version__)
print (os.getcwd())
np.set_printoptions(threshold=sys.maxsize)

IPython version: 8.7.0
numpy version: 1.23.5
scikit-learn version: 1.2.2
matplotlib version: 3.5.1
/home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem


Import the critical news Dataset, and explore its structure and data.

In [24]:
CACHE_NAME = "critical-events.pkz"
TRAIN_FOLDER = "critical-events-train"
TEST_FOLDER = "critical-events-test"
archive_path = "/home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem/data/"
target_dir = "/home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem/data/USGS"
archive_name = "USGS.zip"
if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
patoolib.extract_archive(archive_path+archive_name, outdir=archive_path)
train_path = os.path.join(target_dir, TRAIN_FOLDER)
test_path = os.path.join(target_dir, TEST_FOLDER)
cache_path = target_dir
cache = dict(
        train=load_files(train_path, encoding="latin1"),
        test=load_files(test_path, encoding="latin1"),
    )
with suppress(FileNotFoundError):
        os.remove(cache_path+"_"+CACHE_NAME)
compressed_content = codecs.encode(pickle.dumps(cache), "zlib_codec")
with open(cache_path+"_"+CACHE_NAME, "wb") as f:
    f.write(compressed_content)

#print(cache)

INFO patool: Extracting /home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem/data/USGS.zip ...
INFO patool: running /usr/bin/7zz x -o/home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem/data/ -- /home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem/data/USGS.zip
INFO patool:     with input=''
INFO patool: ... /home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem/data/USGS.zip extracted to `/home/ubuntu/jupyter/Python/MachineLearning/CriticalEventsSystem/data/'.


In [3]:
# train or test datasets
news = cache['train']
news_test = cache['test']

Let's explore the dataset structure:

In [4]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
print (type(news.data))
#print (news.data[0])
#print (news.filenames)
print (len(news.filenames))
print (news.target_names)
print (len(news.target_names))
print (news.target)
print (len(news.target))

<class 'list'>
177
['biology', 'citizen', 'climate', 'coasts', 'ecosystems', 'energy', 'geology', 'information', 'maps', 'methods_analyses', 'minerals', 'natural_hazards', 'ocean', 'other', 'planetary_science', 'science_technology', 'water']
17
[0 5 2 0 5 5 0 5 5 2 5 5 0 2 5 2 0 0 0 0 2 0 0 5 0 5 5 2 5 5 0 0 5 0 0 0 5
 2 2 5 0 0 2 2 5 0 5 2 5 0 5 5 0 5 0 5 5 5 2 5 5 0 2 5 2 5 5 2 0 0 2 2 0 0
 5 2 5 5 5 5 5 5 5 0 5 2 2 5 5 2 0 0 2 0 5 0 5 0 0 2 0 2 2 0 5 5 0 2 2 2 5
 0 5 5 0 0 2 5 0 0 5 5 0 0 0 5 2 0 5 5 0 5 5 0 0 5 0 5 5 0 0 5 0 5 5 5 0 5
 5 5 0 2 2 2 5 5 5 2 2 0 2 5 5 0 0 5 2 2 2 0 0 0 5 2 5 0 5]
177


If you look at, say, the first instance, you will see the content of a newsgroup message, and you can get its corresponding category:

In [6]:
#print (news.data[0])
print (news.target[0], news.target_names[news.target[0]])

0 biology


Let's build the training and testing datasets:

In [7]:
SPLIT_PERC = 0.75
split_size = int(len(news.data)*SPLIT_PERC)
X_train = news.data[:split_size]
X_test = news.data[split_size:]
y_train = news.target[:split_size]
y_test = news.target[split_size:]

#X_train=news.data
#X_test=news_test.data
#y_train=news.target
#y_test=news_test.target

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))


132
45
132
45


This function will serve to perform and evaluate a cross validation:

In [8]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

def evaluate_cross_validation(clf, X, y, K):
    # create a k-fold croos validation iterator of k=5 folds
    cv = KFold(K, shuffle=True, random_state=0)
    # by default the score used is the one returned by score method of the estimator (accuracy)
    scores = cross_val_score(clf, X, y, cv=cv)
    print (scores)
    print (("Mean score: {0:.3f} (+/-{1:.3f})").format(
        np.mean(scores), sem(scores)))

Our machine learning algorithms can work only on numeric data, so our next step will be to convert our text-based dataset to a numeric dataset. Currently we only have one feature, the text content of the message; we need some function that transforms a text into a meaningful set of numeric features. Intuitively one could try to look at which are the words (or more precisely, tokens, including numbers or punctuation signs) that are used in each of the text categories, and try to characterize each category with the frequency distribution of each of those words. The sklearn. feature_extraction.text module has some useful utilities to build numeric feature vectors from text documents.

If you look inside the sklearn.feature_extraction.text module, you will find three different classes that can transform text into numeric features: CountVectorizer, HashingVectorizer, and TfidfVectorizer. The difference between them resides in the calculations they perform to obtain the numeric features. CountVectorizer basically creates a dictionary of words from the text corpus. Then, each instance is converted to a vector of numeric features where each element will be the count of the number of times a particular word appears in the document. HashingVectorizer, instead of constricting and maintaining the dictionary in memory, implements a hashing function that maps tokens into feature indexes, and then computes the count as in CountVectorizer. TfidfVectorizer works like the CountVectorizer, but with a more advanced calculation called Term Frequency Inverse Document Frequency (TF-IDF). This is a statistic for measuring the importance of a word in a document or corpus. Intuitively, it looks for words that are more frequent in the current document, compared with their frequency in the whole corpus of documents. You can see this as a way to normalize the results and avoid words that are too frequent, and thus not useful to characterize the instances.

We will create a Naïve Bayes classifier that is composed of a feature vectorizer and the actual Bayes classifier. We will use the MultinomialNB class from the sklearn.naive_bayes module. 


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer

clf_1 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])
clf_2 = Pipeline([
    ('vect', HashingVectorizer(alternate_sign=False)),
    ('clf', MultinomialNB()),
])
clf_3 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

In [10]:
clfs = [clf_1, clf_2, clf_3]
for clf in clfs:
    evaluate_cross_validation(clf, news.data, news.target, 5)


[0.63888889 0.72222222 0.71428571 0.71428571 0.71428571]
Mean score: 0.701 (+/-0.016)
[0.52777778 0.38888889 0.28571429 0.42857143 0.57142857]
Mean score: 0.440 (+/-0.051)
[0.58333333 0.5        0.34285714 0.6        0.65714286]
Mean score: 0.537 (+/-0.055)


The default regular expression: ur"\b\w\w+\b" considers alphanumeric characters and the underscore. Perhaps also considering the slash and the dot could improve the tokenization, and begin considering tokens as Wi-Fi and site.com. The new regular expression could be: ur"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b". If you have queries about how to define regular expressions, please refer to the Python re module documentation. Let's try our new classifier:

In [11]:
clf_4 = Pipeline([
    ('vect', TfidfVectorizer(
               token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
    )),
    ('clf', MultinomialNB()),
])

clf_14 = Pipeline([
    ('vect', CountVectorizer(
               token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
               
    )),
    ('clf', MultinomialNB()),
])


In [12]:
evaluate_cross_validation(clf_4, news.data, news.target, 5)
evaluate_cross_validation(clf_14, news.data, news.target, 5)

[0.63888889 0.52777778 0.34285714 0.6        0.68571429]
Mean score: 0.559 (+/-0.060)
[0.66666667 0.66666667 0.65714286 0.74285714 0.71428571]
Mean score: 0.690 (+/-0.017)


Another parameter that we can use is stop_words: this argument allows us to pass a list of words we do not want to take into account, such as too frequent words, or words we do not a priori expect to provide information about the particular topic. Let's try to improve performance filtering the stop words:

In [13]:
def get_stop_words():
    result = set()
    for line in open('data/stopwords_en.txt', 'r').readlines():
        result.add(line.strip())
    return result

In [14]:
stop_words = get_stop_words()
print(type(stop_words))
stop_words=list(stop_words)
print(type(stop_words))
print(len(stop_words))
print (stop_words)


<class 'set'>
<class 'list'>
318
['please', 'another', 'many', 'side', 'towards', 'serious', 'mostly', 'back', 'except', 'only', 'bottom', 'several', 'wherein', 'her', 'anything', 'herein', 'was', 'itself', 'when', 'fire', 'eg', 'has', 'nobody', 'against', 'have', 'she', 'name', 'off', 'whole', 'somehow', 'him', 'never', 'while', 'empty', 'de', 'sometime', 'same', 'third', 'also', 'ourselves', 'whereupon', 'hence', 'besides', 'done', 'of', 'me', 'not', 'since', 'whenever', 'i', 'around', 'system', 'be', 'thin', 'thick', 'full', 'to', 'which', 'became', 'six', 'becoming', 'go', 'nothing', 'most', 'with', 'your', 'there', 'themselves', 'but', 'if', 'enough', 'seem', 'interest', 'someone', 'one', 'anyone', 'below', 'onto', 'whatever', 'none', 'himself', 'is', 'amoungst', 'must', 'five', 'from', 'them', 'whereas', 'so', 'during', 'throughout', 'mill', 'otherwise', 'were', 'yourselves', 'well', 'therein', 'become', 'us', 'thence', 'thereafter', 'been', 'fill', 'or', 'myself', 'although', 't

In [15]:
clf_5 = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",  
    )),
    ('clf', MultinomialNB()),
])
clf_15 = Pipeline([
    ('vect', CountVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",  
    )),
    ('clf', MultinomialNB()),
])

In [16]:
evaluate_cross_validation(clf_5, news.data, news.target, 5)
evaluate_cross_validation(clf_15, news.data, news.target, 5)

[0.69444444 0.61111111 0.51428571 0.6        0.62857143]
Mean score: 0.610 (+/-0.029)
[0.66666667 0.66666667 0.65714286 0.74285714 0.71428571]
Mean score: 0.690 (+/-0.017)


Try to improve by adjusting the alpha parameter on the MultinomialNB classifier:

In [17]:
clf_7 = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",         
    )),
    ('clf', MultinomialNB(alpha=0.01)),
])
clf_17 = Pipeline([
    ('vect',CountVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",         
    )),
    ('clf', MultinomialNB(alpha=1)),
])

In [18]:
evaluate_cross_validation(clf_7, news.data, news.target, 5)
evaluate_cross_validation(clf_17, news.data, news.target, 5)

[0.63888889 0.61111111 0.68571429 0.68571429 0.74285714]
Mean score: 0.673 (+/-0.023)
[0.66666667 0.66666667 0.65714286 0.74285714 0.71428571]
Mean score: 0.690 (+/-0.017)


The results had an important boost from 0.61 to 0.67, pretty good. At this point, we could continue doing trials by using different values of alpha or doing new modifications of the vectorizer.

If we decide that we have made enough improvements in our model, we are ready to evaluate its performance on the testing set.

In [19]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print ("Accuracy on training set:")
    print (clf.score(X_train, y_train))
    print ("Accuracy on testing set:")
    print (clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print ("Classification Report:")
    print (metrics.classification_report(y_test, y_pred))
    print ("Confusion Matrix:")
    print (metrics.confusion_matrix(y_test, y_pred))

In [20]:
train_and_evaluate(clf_17, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.9545454545454546
Accuracy on testing set:
0.7777777777777778
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        15
           2       0.55      0.60      0.57        10
           5       0.83      0.75      0.79        20

    accuracy                           0.78        45
   macro avg       0.75      0.76      0.75        45
weighted avg       0.78      0.78      0.78        45

Confusion Matrix:
[[14  0  1]
 [ 2  6  2]
 [ 0  5 15]]


As we can see, we obtained very good results, and as we would expect, the accuracy in the training set is quite better than in the testing set. We may expect, in new unseen instances, an accuracy of around 0.78.

If we look inside the vectorizer, we can see which tokens have been used to create our dictionary:

In [21]:
clf_17.named_steps['vect'].get_feature_names_out()

array(['.anincreaseof', '.thisincludesbudgetincreasesof', '000-year',
       '08_news_releases', '0x55c1e7a8d65ee306', '1.2billion',
       '1.3millionincreaseforstreamflowinformation',
       '1.5millionincreasethatsupportsscienceforpugetsound',
       '1.6billioninprotection',
       '1.6millionincreasetothemineralresourcesprogramforidentifyingandevaluatingnewsourcesofcriticalmineralsandfornewscienceandtoolstoreducetheimpactsofmineralsextraction',
       '1.6milliontostudythedeclineofinsects', '10-fold', '10-member',
       '10-square',
       '10.7millionincreaseabovethefy2016enactedlevelforatotalof',
       '100-year', '100-year-flood-areas-maui-hawaii', '111208.html',
       '113-mile-long',
       '13.7millionabovethefy2016enactedlevelforatotalof', '136-year',
       '14-year', '14-year-study',
       '14.5millionabovethefy2015enactedlevelforsciencetosupportsustainablewatermanagement.',
       '14379-frac-sand-sources-in-the-united-states.html', '15-year',
       '15.6milliontoex

In [22]:
print (len(clf_17.named_steps['vect'].get_feature_names_out()))

8324
